This notebook analyses the [Yelp Dataset](https://www.yelp.com/dataset) using a LLM agent as a copilot. First we'll reorganize the data and put it in pandas a dataframe. This is in order to get the data organized in a format where it is easier to instruct the LLM how to analyze it. Alternatively we could try to get the LLM agent to restructure the data for us. Feel free to give this a shot!

In [1]:
import json
import pandas as pd
from typing import Dict, List


def read_json_data(filepath: str):
    data = []
    print('loading json')
    with open(filepath, 'r') as jsonfile:
        for line in jsonfile.readlines():
            data.append( json.loads(line) )
    return data

def extract_non_nested_fields(data: List[Dict]):
    record = data[0]
    columns = set([c for c in record if type(record[c]) != dict and c != 'hours'])
    print('reformatting json')
    records = []
    return [
        {key: record[key] for key in record if key in columns }
        for record in data
    ]

def load_json_to_df(filepath: str):
    data = read_json_data(filepath)
    data = extract_non_nested_fields(data)
    print('loading data into dataframe')
    return pd.DataFrame(data)
        

In [2]:
filepath = 'data/yelp_dataset/yelp_academic_dataset_business.json'

In [3]:
business_df = load_json_to_df('data/yelp_dataset/yelp_academic_dataset_business.json')

loading json
reformatting json
loading data into dataframe


Lets take al ook at the dataframes content.

In [4]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food"


We will use a custom `dataframeAnalysis` tool to enable our langchain agent to perform operations on the Pandas dataframes in our python environment and store the results. This tool requires all input dataframes which will be used for analysis to be registered with the `chat_agent` by calling the function `chat_agent.set_inputs`. Each key in the dataframe will correspond to the dataframes name. The value for each key will be a dictionary containing the dataframe itself under a `df` key and a description of the dataframe under a `description` key. 

For example:
```
import chat_agent

chat_agent.set_inputs(
    {
        'business_df': {
            'df': business_df,
            'description': 'this is a dataframe containing information about businesses'
        }
    }
)
```

In [5]:
import chat_agent
agent_inputs = {
    'business_df': {'value': business_df, 'description': 'this is a dataframe containing information about businesses'}
}
chat_agent.set_inputs(agent_inputs)

In [6]:
%%chat_agent
Give me all of the businesses that are in PA.

prompt is 
Give me all of the businesses that are in PA.

When using the dataframeAnalysis tool you may assume that you have access to the following variables when writing the code:
            business_df - this is a dataframe containing information about businesses


> Entering new AgentExecutor chain...
description extracted I need to filter the business_df dataframe to only include businesses in PA
 I need to filter the business_df dataframe to only include businesses in PA
Action: dataframeAnalysis
Action Input: business_df[business_df['state'] == 'PA']cleaned code
business_df[business_df['state'] == 'PA']

Observation: Answer has been successfully derived. Key: result1
Thought:description extracted I now need to extract the business names from the result1 dataframe
 I now need to extract the business names from the result1 dataframe
Action: dataframeAnalysis
Action Input: result1['name'].tolist()cleaned code
result1['name'].tolist()

Observation: Answer has been successfully deri

'The businesses in PA are: result2'

In [7]:
chat_agent.get_result('result1')

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food"
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"Sushi Bars, Restaurants, Japanese"
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"Korean, Restaurants"
25,PSo_C1Sfa13JHjzVNW6ziQ,Indian Walk Veterinary Center,662 Durham Rd,Newtown,PA,18940,40.273493,-74.987496,5.0,15,1,"Pet Services, Pet Groomers, Pets, Veterinarians"
...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,1,"Restaurants, Sandwiches, Convenience Stores, C..."
150326,JDM7kiCEmeBc_5W-z6eZZw,2601 Parkway Apartments,2601 Pennsylvania Ave,Philadelphia,PA,19130,39.968806,-75.180608,3.0,6,1,"Real Estate, Apartments, Home Services"
150329,9U1Igcpe954LoWZRmNc-zg,Hand & Stone Massage And Facial Spa,"1100 S Columbus Blvd, Ste 24",Philadelphia,PA,19147,39.932756,-75.144504,3.0,32,1,"Day Spas, Beauty & Spas, Skin Care, Massage"
150334,LJ4GjQ1HL6kqvIPpNUNNaQ,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,PA,19103,39.945966,-75.169666,4.5,39,1,"Health & Medical, Yoga, Shopping, Naturopathic..."


In [11]:
chat_agent.get_result('result2')[:10]

['St Honore Pastries',
 'Perkiomen Valley Brewery',
 'Tuna Bar',
 'BAP',
 'Indian Walk Veterinary Center',
 'Bar One',
 'DeSandro on Main',
 'DanceLine',
 'Ardmore Pizza',
 'Craft Hall']

In [10]:
%%chat_agent
using result1 tell me which business has the greatest number of reviews

prompt is 
using result1 tell me which business has the greatest number of reviews

When using the dataframeAnalysis tool you may assume that you have access to the following variables when writing the code:
            business_df - this is a dataframe containing information about businesses
result1 - I need to filter the business_df dataframe to only include businesses in PA
result2 - I now need to extract the business names from the result1 dataframe


> Entering new AgentExecutor chain...
description extracted I need to group the businesses by name and count the number of reviews
 I need to group the businesses by name and count the number of reviews
Action: dataframeAnalysis
Action Input: result2.groupby('name').count()['review_id']cleaned code
result2.groupby('name').count()['review_id']


AttributeError: 'list' object has no attribute 'groupby'